In [1]:
import os,sys
from typing import List, Dict
from tqdm import tqdm
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, roc_auc_score
import lightgbm as lgb
import matplotlib.pyplot as plt
from my_lgbm import *

# Code 경로 추가
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath(""))))
print(sys.path[-1])

C:\Users\Minari\Documents\GitHub\level1-classificationinmachinelearning-recsys-06


In [2]:
df = pd.read_csv('scaled_eda.csv')
df

,ID,_type,target,scaled_log_hashrate,scaled_log_open_interest,scaled_log_coinbase_premium_index,scaled_funding_rates,scaled_estimated_block_reward,scaled_liquidation_diff,scaled_log_total_liquidation,scaled_log_total_taker_volume,scaled_utxo_count,scaled_total_transactions_count,taker_buy_sell_ratio,moving_avg_scaled_log_total_volume,open_interest_diff
0,2023-01-01 00:00:00,train,2.0,0.343899,-2.074545,-1.004418,-0.190599,-2.283932,-0.013822,-1.407440,-1.507090,-1.216481,-0.616412,0.904774,-1.507090,NaN
1,2023-01-01 01:00:00,train,1.0,-0.479569,-2.055443,-0.920481,-0.190611,-2.271191,-0.020702,-1.089259,-1.998687,-1.216149,-1.202730,1.655721,-1.752889,1.733962e+07
2,2023-01-01 02:00:00,train,1.0,0.039981,-2.057519,-0.982775,-0.190679,-2.285286,-0.013936,-1.414659,-2.337608,-1.216188,-1.232124,1.027512,-1.947795,-1.886794e+06
3,2023-01-01 03:00:00,train,1.0,-0.312311,-2.059965,-0.995009,-0.188985,-2.286295,-0.008301,-1.132860,-2.043796,-1.215593,-1.263915,0.874477,-1.971795,-2.221901e+06
4,2023-01-01 04:00:00,train,2.0,-0.060107,-2.052255,-1.026415,-0.083030,-2.282372,-0.010506,-1.228120,-2.015961,-1.214962,-1.087030,0.966796,-1.980628,7.007341e+06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11547,2024-04-26 03:00:00,test,NaN,-0.369730,0.534373,-0.235206,NaN,-1.782316,-0.032620,-1.383570,NaN,2.350929,-0.872964,NaN,NaN,-2.193716e+07
11548,2024-04-26 04:00:00,test,NaN,0.191102,NaN,-0.472610,NaN,-1.918750,0.052758,-0.438363,NaN,2.352974,NaN,NaN,NaN,NaN
11549,2024-04-26 05:00:00,test,NaN,0.191102,NaN,-0.157720,NaN,-1.981323,-0.088241,-0.412978,NaN,2.355665,NaN,NaN,NaN,NaN
11550,2024-04-26 06:00:00,test,NaN,-0.136964,NaN,-0.247183,NaN,-2.150662,-0.051507,-0.990694,NaN,2.355852,NaN,NaN,NaN,NaN


In [3]:
df.isna().sum()

ID                                       0
_type                                    0
target                                2792
scaled_log_hashrate                      0
scaled_log_open_interest                 4
scaled_log_coinbase_premium_index        7
scaled_funding_rates                    32
scaled_estimated_block_reward           24
scaled_liquidation_diff                  0
scaled_log_total_liquidation             0
scaled_log_total_taker_volume           31
scaled_utxo_count                        1
scaled_total_transactions_count          4
taker_buy_sell_ratio                    31
moving_avg_scaled_log_total_volume       6
open_interest_diff                       5
dtype: int64

In [4]:
df['ID'] = pd.to_datetime(df['ID'])

In [5]:
train_df = df.loc[df["_type"]=="train"].drop(columns=["_type", 'scaled_utxo_count', 'scaled_log_total_taker_volume'])
test_df = df.loc[df["_type"]=="test"].drop(columns=["_type", 'scaled_utxo_count', 'scaled_log_total_taker_volume'])

In [6]:

# train_test_split 으로 valid set, train set 분리
x_train, x_valid, y_train, y_valid = train_test_split(
    train_df.drop(["target", "ID"], axis = 1), 
    train_df["target"].astype(int), 
    test_size=0.2,
    random_state=42,
)

# lgb dataset
train_data = lgb.Dataset(x_train, label=y_train)
valid_data = lgb.Dataset(x_valid, label=y_valid, reference=train_data)

# lgb params
params = {
    "boosting_type": "gbdt",
    "objective": "multiclass",
    "metric": "multi_logloss",
    "num_class": 4,
    "num_leaves": 50,
    "learning_rate": 0.05,
    "n_estimators": 30,
    "random_state": 42,
    "verbose": 0,
}

# lgb train
lgb_model = lgb.train(
    params=params,
    train_set=train_data,
    valid_sets=valid_data,
)

# lgb predict
y_valid_pred = lgb_model.predict(x_valid)
y_valid_pred_class = np.argmax(y_valid_pred, axis = 1)

# score check
accuracy = accuracy_score(y_valid, y_valid_pred_class)
auroc = roc_auc_score(y_valid, y_valid_pred, multi_class="ovr")

print(f"acc: {accuracy}, auroc: {auroc}")

C:\Users\Minari\anaconda3\envs\bProject01\Lib\site-packages\lightgbm\engine.py:172: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


acc: 0.4235159817351598, auroc: 0.6178621518210089


In [7]:
train_merged_df_with_target = x_train.copy()
train_merged_df_with_target['close'] = y_train

# 피어슨 상관계수 계산
correlation_matrix = train_merged_df_with_target.corr()

# 타겟과 각 피처 간의 상관계수만 추출 후 반환
correlation_matrix['close'].drop('close').sort_values(ascending=False)

scaled_liquidation_diff               0.025992
scaled_log_coinbase_premium_index     0.019331
scaled_log_total_liquidation          0.012603
scaled_log_hashrate                   0.007928
moving_avg_scaled_log_total_volume    0.002084
scaled_total_transactions_count      -0.006974
scaled_estimated_block_reward        -0.013124
scaled_log_open_interest             -0.015394
scaled_funding_rates                 -0.019453
open_interest_diff                   -0.021962
taker_buy_sell_ratio                 -0.041525
Name: close, dtype: float64

In [8]:
f_import = lgb_model.feature_importance(importance_type='gain')
f_names = x_train.columns

ser = pd.Series(f_import, index=f_names, name='gain')

ser = ser / f_import.sum()

ser.sort_values(ascending=False, inplace=True)

ser

scaled_estimated_block_reward         0.119306
moving_avg_scaled_log_total_volume    0.115155
scaled_log_coinbase_premium_index     0.110983
scaled_log_total_liquidation          0.096747
taker_buy_sell_ratio                  0.094375
open_interest_diff                    0.090116
scaled_liquidation_diff               0.086745
scaled_log_open_interest              0.080833
scaled_total_transactions_count       0.076976
scaled_funding_rates                  0.069931
scaled_log_hashrate                   0.058832
Name: gain, dtype: float64

In [9]:
f_import = lgb_model.feature_importance(importance_type='split')
f_names = x_train.columns

ser = pd.Series(f_import, index=f_names, name='split')

ser.sort_values(ascending=False, inplace=True)

ser

scaled_log_coinbase_premium_index     728
scaled_estimated_block_reward         639
taker_buy_sell_ratio                  637
open_interest_diff                    569
moving_avg_scaled_log_total_volume    559
scaled_total_transactions_count       509
scaled_log_open_interest              472
scaled_log_total_liquidation          470
scaled_funding_rates                  462
scaled_liquidation_diff               429
scaled_log_hashrate                   406
Name: split, dtype: int32

In [10]:
submission_df: pd.DataFrame = pd.read_csv(os.path.join('../../data/', "test.csv"))

In [11]:
y_test_pred = lgb_model.predict(test_df.drop(["target", "ID"], axis = 1))
y_test_pred_class = np.argmax(y_test_pred, axis = 1)

In [12]:
submission_df = submission_df.assign(target = y_test_pred_class)
submission_df.to_csv("output3.csv", index=False)

In [17]:
output2_df = pd.read_csv('output2.csv') #scaled
output3_df = pd.read_csv('output3.csv')   #scaled drop 'scaled_utxo_count', 'scaled_log_total_taker_volume'

print(len(output2_df.compare(output3_df)))

705


In [18]:
output3_df['target'].value_counts()

target
2    1436
1    1321
3      22
0      13
Name: count, dtype: int64

In [19]:
output3_df['target']
output3_df

,ID,target
0,2024-01-01 00:00:00,1
1,2024-01-01 01:00:00,1
2,2024-01-01 02:00:00,2
3,2024-01-01 03:00:00,2
4,2024-01-01 04:00:00,2
...,...,...
2787,2024-04-26 03:00:00,1
2788,2024-04-26 04:00:00,1
2789,2024-04-26 05:00:00,1
2790,2024-04-26 06:00:00,1
